In [1]:
using JuMP
using Gurobi
import Printf

In [2]:
G = [
    0 3 2 2 0 0 0 0
    0 0 0 0 5 1 0 0
    0 0 0 0 1 3 1 0
    0 0 0 0 0 1 0 0
    0 0 0 0 0 0 0 4
    0 0 0 0 0 0 0 2
    0 0 0 0 0 0 0 4
    0 0 0 0 0 0 0 0
]
n = size(G, 1)
M = -sum(G)

-29

In [3]:
function optimal_flows(x)
    return [(i, j) => x[i, j] for i in 1:n for j in 1:n if x[i, j] > 0]
end

optimal_flows (generic function with 1 method)

In [4]:
model = Model(Gurobi.Optimizer)
set_silent(model)
@variable(model, x[1:n, 1:n], Bin)
@variable(model, θ >= M)
@constraint(model, sum(x) <= 11)
@objective(model, Min, 0.1 * sum(x) + θ)
model

Set parameter Username
Set parameter LicenseID to value 2646351
Academic license - for non-commercial use only - expires 2026-04-02


A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 65
├ num_constraints: 66
│ ├ AffExpr in MOI.LessThan{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 64
└ Names registered in the model
  └ :x, :θ

In [7]:
subproblem = Model(Gurobi.Optimizer)
set_silent(subproblem)
# We need to turn presolve off so that HiGHS will return an infeasibility certificate.
# set_attribute(subproblem, "presolve", "off")
@variable(subproblem, x_copy[i in 1:n, j in 1:n])
@variable(subproblem, y[1:n, 1:n] >= 0)
@constraint(subproblem, sum(y) >= 1)  # <--- THIS IS NEW
@constraint(subproblem, [i = 1:n, j = 1:n], y[i, j] <= G[i, j] * x_copy[i, j])
@constraint(subproblem, [i = 2:n-1], sum(y[i, :]) == sum(y[:, i]))
@objective(subproblem, Min, -sum(y[1, :]))
subproblem

Set parameter Username
Set parameter LicenseID to value 2646351
Academic license - for non-commercial use only - expires 2026-04-02


A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 128
├ num_constraints: 135
│ ├ AffExpr in MOI.EqualTo{Float64}: 6
│ ├ AffExpr in MOI.GreaterThan{Float64}: 1
│ ├ AffExpr in MOI.LessThan{Float64}: 64
│ └ VariableRef in MOI.GreaterThan{Float64}: 64
└ Names registered in the model
  └ :x_copy, :y

In [8]:
function solve_subproblem_with_feasibility(model, x)
    fix.(model[:x_copy], x)
    optimize!(model)
    if is_solved_and_feasible(model; dual = true)
        return (
            is_feasible = true,
            obj = objective_value(model),
            y = value.(model[:y]),
            π = reduced_cost.(model[:x_copy]),
        )
    end
    return (
        is_feasible = false,
        v = dual_objective_value(model),
        u = reduced_cost.(model[:x_copy]),
    )
end

solve_subproblem_with_feasibility (generic function with 1 method)

In [10]:
MAXIMUM_ITERATIONS = 100
println("Iteration  Lower Bound  Upper Bound          Gap")
for k in 1:MAXIMUM_ITERATIONS
    optimize!(model)
    assert_is_solved_and_feasible(model)
    lower_bound = objective_value(model)
    x_k = value.(x)
    ret = solve_subproblem_with_feasibility(subproblem, x_k)
    if ret.is_feasible
        # Benders Optimality Cuts
        upper_bound = (objective_value(model) - value(θ)) + ret.obj
        gap = abs(upper_bound - lower_bound) / abs(upper_bound)
        print_iteration(k, lower_bound, upper_bound, gap)
        if gap < ABSOLUTE_OPTIMALITY_GAP
            println("Terminating with the optimal solution")
            break
        end
        @constraint(model, θ >= ret.obj + sum(ret.π .* (x .- x_k)))
    else
        # Benders Feasibility Cuts
        cut = @constraint(model, ret.v + sum(ret.u .* (x .- x_k)) <= 0)
        @info "Adding the feasibility cut $(cut)"
    end
end

Iteration  Lower Bound  Upper Bound          Gap


MathOptInterface.ResultIndexBoundsError{MathOptInterface.DualObjectiveValue}: Result index of attribute MathOptInterface.DualObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.